In [1]:
import pandas as pd
import string
from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


# BW1484 Core I Output

In [2]:
k_ref = 1.00020

### Load and parse the file

In [5]:
import re

# name of the file
out_file='bw1484_small.out' #$1
f = open('/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/'+out_file, 'r')

## Pairs of output variables and their keywords that always occur and only once
keys = ['problem_name', 'eigen_solver', 'eq_set', 'num cells I', 'num cells J', 
        'num cells K', 'num_blocks_i', 'num_blocks_j', 'num_z_blocks', 'DoF', 
        'L2_tolerance', 'k_tolerance', 'first_group', 'last_group', 'mesh ', 'num_sets', 
        'num upscatter groups', 'mg_preconditioner', 'relax_count', 'num_v_cycles', 
        'relax_weight']
vals = {key: None for key in keys}

for line in f:
    # look for things that always occur and only once
    for key in vals: 
        if re.search(key, line):
            data = line.split(key)
            vals[key] = data[1].strip() 
            print key, vals[key]   
        
vals['num_groups'] = int(vals['last_group']) - int(vals['first_group']) + 1
print 'num_groups', vals['num_groups']

# ones needing special handling: 'Pn_order', 'azimuthals_octant', 'polars_octant', 'tolerance', 

# if multilevel_quad_db,
# Pn_order,
# azimuthals_octant,
# polars_octant,
# if upscatter_db; tolerance,



Pn_order 1
first_group 0
last_group 7
mesh  2
num_blocks_i 16
num_blocks_j 16
num_sets 2
num_v_cycles 1
num_z_blocks 7
relax_count 1
relax_weight 1
tolerance 0.001
mesh  0
eigen_solver rayleigh_quotient
eq_set ld
mg_preconditioner Multilevel
problem_name out_bw1484_RQI_sn
tolerance 0.001
L2_tolerance 0.001
k_tolerance 0.001
tolerance 0.001
Pn_order 1
azimuthals_octant 2
polars_octant 2
Pn_order 1
azimuthals_octant 2
polars_octant 2
tolerance 0.001
num cells I 120
num cells J 120
num cells K 84
num upscatter groups 4
DoF 4954521600
num_groups 8
